#### 断点续训，存取模型
##### 第一种方式
- 模型保存
   ~~~python
   tf.keras.callbacks.ModelCheckpoint(
       filepath=路径文件名,
       save_weight_only=True/False,
       save_best_only=True/False,
   )
   history=model.fit(callback=[])
   ~~~
- 读取模型
  ~~~python
  load_weights(路径文件名)
  checkpoint_save_path="mnist.ckpt"
  model=创建一个一模一样的模型
  #加载参数数据
  model.load_weights(checkpoint_save_path)
  model.evaluate()#模型评估
  ~~~
##### 第二种方式
- 模型权重参数保存
  ~~~python
   model.save_weights(model_name)
  ~~~
- 加载最后一次模型参数数据
  ~~~python
   model=tf.train.latest_checkpoint(checkpoint_dir=checkpointDir)
  ~~~
##### 第三种方式（比较常见）
- 保存整个模型
  ~~~python
   model.save(路径)
  ~~~
- 加载h5模型
  ~~~python
  model=tf.keras.models.load_model(路径)
  ~~~
 #### 读取模型后进行预测
 ~~~python
 model.predict(输入特征)
~~~

In [1]:
import os
import tensorflow as tf
import numpy as np
from sklearn import datasets
from tensorflow.keras import Model

In [2]:
#### 鸢尾花数据加载
x_train=datasets.load_iris().data
y_train=datasets.load_iris().target
np.random.seed(116)
np.random.shuffle(x_train)
np.random.seed(116)
np.random.shuffle(y_train)

In [3]:
class my_model(Model):
    def __init__(self):
        super(my_model,self).__init__()
        self.d=tf.keras.layers.Dense(3,activation='softmax',kernel_regularizer=tf.keras.regularizers.l2())
    def call(self,x):
        return self.d(x)
model=my_model()
path="./checkpoint/"
if not os.path.exists(path):
    os.mkdir(path)
callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=path+"my_model.ckpt",
    save_bast_only=True,
    save_weights_only=True
)
model.compile(
    optimizer=tf.keras.optimizers.SGD(lr=0.01),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['sparse_categorical_accuracy']
)
#model.fit(x_train,y_train,batch_size=32,epochs=10,validation_split=0.2,validation_freq=20,callbacks=[callback])
model.fit(x_train,y_train,batch_size=32,epochs=10,validation_split=0.2,validation_freq=20)

Train on 120 samples, validate on 30 samples
Epoch 1/10
120/120 [==============================] - 0s 1ms/sample - loss: 2.8606 - sparse_categorical_accuracy: 0.1750
Epoch 2/10
120/120 [==============================] - 0s 58us/sample - loss: 2.0261 - sparse_categorical_accuracy: 0.0083
Epoch 3/10
120/120 [==============================] - 0s 75us/sample - loss: 1.7041 - sparse_categorical_accuracy: 0.1250
Epoch 4/10
120/120 [==============================] - 0s 91us/sample - loss: 1.5302 - sparse_categorical_accuracy: 0.2667
Epoch 5/10
120/120 [==============================] - 0s 50us/sample - loss: 1.4352 - sparse_categorical_accuracy: 0.2917
Epoch 6/10
120/120 [==============================] - 0s 125us/sample - loss: 1.3802 - sparse_categorical_accuracy: 0.3083
Epoch 7/10
120/120 [==============================] - 0s 67us/sample - loss: 1.3380 - sparse_categorical_accuracy: 0.3417
Epoch 8/10
120/120 [==============================] - 0s 50us/sample - loss: 1.2989 - sparse_categori

In [4]:
###将保存的模型参数进行加载
model.load_weights(path+"my_model.ckpt")
model.evaluate(x_train,y_train)

150/150 [==============================] - 0s 377us/sample - loss: 1.7807 - sparse_categorical_accuracy: 0.3333


[1.7807463804880779, 0.33333334]

In [5]:
#### 将模型加载成为h5模型
if not os.path.exists("./h5"):
    os.mkdir("./h5")
model.save("./h5/")
models=tf.keras.models.load_model("./h5/")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./h5/assets


In [6]:
models.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  15        
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


In [7]:
models.evaluate(x_train,y_train)

150/150 [==============================] - 0s 412us/sample - loss: 1.7807 - sparse_categorical_accuracy: 0.3333


[1.780746405919393, 0.33333334]